<a href="https://colab.research.google.com/github/sebug/hype-combiner/blob/main/LinkedInLearning/leprogramme.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Challenge
Have a look at [Le Programme](https://leprogramme.ch/) - this is an events agenda for Geneva. We want to query it using natural language. The idea is to construct the search URL, and let you go there, but only if we constructed something valid. We can double check the combination between events returned and search query by filtering the full list as well - listing the events on the main page that match, and then finding a search clause that gives the most similar results.

I don't know whether this works yet, and will be exploring a lot.

In [ ]:
print("Test content")

Test content
